# Attribution CIs for Integrated Gradients

(Average absolute) integrated gradients, designed specifically for the linear setting

In [1]:
import numpy as np
import torch
from sklearn.linear_model import LinearRegression
from uncertainty.bootstrapCoefficients import bootstrapCis
from uncertainty.analyticLinearRegressionCIs import analyticLinearCis
from uncertainty.DataGeneration import default_data, linearRegression_normal
from uncertainty.glrtTorch import glrtTorchCis, MSE
from uncertainty.torch_linear import TorchLinear
from uncertainty.glrt_stat import bootstrapGLRTcis
from uncertainty.IG_linearModels import integratedGradients_Linear, bootstrapIntegratedGradients_Linear, integratedGradients_LinearAnalytical
from attributionpriors.pytorch_ops import ExpectedGradientsModel

In [2]:
from matplotlib import pyplot as plt
%matplotlib inline

In [3]:
X, y = default_data()

## Test analytical and bootstrap IG CIs

In [4]:
# Sklearn Linear Regression
LR = LinearRegression()
LR.fit(X, y)
print("Coefficients:", LR.coef_)
print("Integrated gradients:", integratedGradients_Linear(LR.coef_, X))

print("Bootstrapping")
lcb_LR, ucb_LR = bootstrapIntegratedGradients_Linear(LinearRegression, X, y, alpha=0.05, replicates=1000)
print("Lower bounds:", lcb_LR, "\nUpper bounds:", ucb_LR)

print("Analytical")
lcb_LR_a, ucb_LR_a = integratedGradients_LinearAnalytical(LR, X, y, alpha=0.05)
print("Lower bounds:", lcb_LR_a, "\nUpper bounds:", ucb_LR_a)

Coefficients: [0.88513371 1.18504362]
Integrated gradients: [0.76678261 1.08949336]
Bootstrapping
Lower bounds: [0.62096489 0.93464367] 
Upper bounds: [0.92118144 1.23273253]
Analytical
Lower bounds: [0.61703249 0.93594248] 
Upper bounds: [0.91653274 1.24304425]


In [5]:
# Torch Linear Regression
TL = TorchLinear(lr=0.3,max_iter=30)
TL.fit(X,y)

print("Coefficients:", TL.coef_)
print("Integrated gradients:", integratedGradients_Linear(TL.coef_, X))

# Takes ~4min
print("Bootstrapping")
lcb_TL, ucb_TL = bootstrapIntegratedGradients_Linear(lambda:TorchLinear(lr=0.3,max_iter=30), X=X, y=y, alpha=0.05, replicates=1000)
print("Lower bounds:", lcb_TL, "\nUpper bounds:", ucb_TL)

print("Analytical")
lcb_TL_a, ucb_TL_a = integratedGradients_LinearAnalytical(TL, X, y, alpha=0.05)
print("Lower bounds:", lcb_TL_a, "\nUpper bounds:", ucb_TL_a)

Coefficients: [0.8851187 1.1850578]
Integrated gradients: [0.76676963 1.08950636]
Bootstrapping
Lower bounds: [0.62688759 0.94106662] 
Upper bounds: [0.91870455 1.22105407]
Analytical
Lower bounds: [0.61701951 0.93595548] 
Upper bounds: [0.91651975 1.24305725]


In [6]:
# Check that Torch IG calculation gives same result as analytic
Rtorch = torch.ones_like(torch.Tensor(X))*X.mean(0).reshape(1,-1)
Rset = torch.utils.data.TensorDataset(Rtorch)
EGM = ExpectedGradientsModel(TL.model,Rset,k=10,random_alpha=False,scale_by_inputs=True)
preds,igs = EGM(torch.Tensor(X),shap_values=True)
print("Torch IG:", igs.abs().mean(0).detach().numpy())

Torch IG: [0.7667699 1.089506 ]


In [7]:
# GLRT method with Torch model
TL = TorchLinear(lr=0.3,max_iter=30)
TL.fit(X,y)
print("GLRT")
lcb_GLRT, ucb_GLRT, lcb_Results, ucb_Results = glrtTorchCis(lambda:TorchLinear(lr=0.3,max_iter=100), X=X, y=y, alpha=0.05,search_kwargs={'lmbds':np.logspace(-10,10,101)},fit_kwargs={'lr':0.3,'max_iter':30})
print("Lower bounds:", lcb_GLRT, "\nUpper bounds:", ucb_GLRT)

GLRT
Lower bounds: [0.41983467 0.72502303] 
Upper bounds: [1.11372757 1.45386302]


In [ ]:
# Manually check Torch GLRT answer
lcb_MSE, ucb_MSE = bootstrapGLRTcis(lambda:TorchLinear(lr=0.3,max_iter=100), X, y, lambda x,y: np.mean((x-y)**2), alpha=0.05)
(mses, attributions, coefs, biases) = ucb_Results[0]
valid_inds = mses<=ucb_MSE
valid_attribs, valid_coefs, valid_biases = attributions[valid_inds], coefs[valid_inds], biases[valid_inds]
max_ind = np.argmax(valid_attribs)
max_coef, max_bias = valid_coefs[max_ind], valid_biases[max_ind]
print("Torch attribution search found the following model:")
print(f'Coefs: {max_coef}, Bias: {max_bias}')
print(f'Corresponding Analytic IG: {integratedGradients_Linear(max_coef,X)}')
print(f'Model MSE {MSE(y,X@max_coef+max_bias):.3f} < MSE UCB {ucb_MSE:.3f}')

## Other examples

In [7]:
# Do an example where the coefficient CI's cross zero:
X, y = linearRegression_normal(beta=np.array([0, 1]).T,
                               cov=np.array([[1, 0.5],[0.5, 1]]),
                               sigma=1,
                               n=200)

In [8]:
LR = LinearRegression()
LR.fit(X, y)
print("Coefficients:", LR.coef_)
lcb_LR_coef, ucb_LR_coef = bootstrapCis(LinearRegression, X, y, alpha=0.05, replicates=1000)
print("Coefficient CI's (analytical)", "\nLower bounds:", lcb_LR_coef, "\nUpper bounds:", ucb_LR_coef)

print("Integrated gradients:", integratedGradients_Linear(LR.coef_, X))

print("Bootstrapping")
lcb_LR, ucb_LR = bootstrapIntegratedGradients_Linear(LinearRegression, X, y, alpha=0.05, replicates=1000)
print("Lower bounds:", lcb_LR, "\nUpper bounds:", ucb_LR)

print("Analytical")
lcb_LR_a, ucb_LR_a = integratedGradients_LinearAnalytical(LR, X, y, alpha=0.05)
print("Lower bounds:", lcb_LR_a, "\nUpper bounds:", ucb_LR_a)

Coefficients: [-0.03269696  0.99281269]
Coefficient CI's (analytical) 
Lower bounds: [-0.18784993  0.81820811] 
Upper bounds: [0.12354142 1.16604283]
Integrated gradients: [0.02412255 0.83430801]
Bootstrapping
Lower bounds: [0.00294276 0.69340201] 
Upper bounds: [0.14072949 0.9716986 ]
Analytical
Lower bounds: [0.         0.69714906] 
Upper bounds: [0.15338195 0.97146696]


In [9]:
# And an example where the coefficient CI's are negative:
X, y = linearRegression_normal(beta=np.array([-1, 1]).T,
                               cov=np.array([[1, 0.5],[0.5, 1]]),
                               sigma=1,
                               n=200)

In [10]:
LR = LinearRegression()
LR.fit(X, y)
print("Coefficients:", LR.coef_)
lcb_LR_coef, ucb_LR_coef = bootstrapCis(LinearRegression, X, y, alpha=0.05, replicates=1000)
print("Coefficient CI's (analytical)", "\nLower bounds:", lcb_LR_coef, "\nUpper bounds:", ucb_LR_coef)

print("Integrated gradients:", integratedGradients_Linear(LR.coef_, X))

print("Bootstrapping")
lcb_LR, ucb_LR = bootstrapIntegratedGradients_Linear(LinearRegression, X, y, alpha=0.05, replicates=1000)
print("Lower bounds:", lcb_LR, "\nUpper bounds:", ucb_LR)

print("Analytical")
lcb_LR_a, ucb_LR_a = integratedGradients_LinearAnalytical(LR, X, y, alpha=0.05)
print("Lower bounds:", lcb_LR_a, "\nUpper bounds:", ucb_LR_a)

Coefficients: [-0.89721292  0.79395396]
Coefficient CI's (analytical) 
Lower bounds: [-1.08906147  0.60376828] 
Upper bounds: [-0.70392245  0.96846779]
Integrated gradients: [0.63735869 0.62667378]
Bootstrapping
Lower bounds: [0.49126259 0.47295379] 
Upper bounds: [0.78294865 0.76584111]
Analytical
Lower bounds: [0.50067477 0.49388333] 
Upper bounds: [0.7740426  0.75946423]
